In [1]:
from ultralytics import YOLO

In [2]:
# Load best run of model
model = YOLO("runs/detect/train/weights/best.pt")

In [3]:
# Make a prediction
prediction = model.predict("../data/images/train/000000.png", save = True)


image 1/1 /Users/langtowl/Desktop/adl-mini-project/notebooks/../data/images/train/000000.png: 224x640 1 Pedestrian, 34.8ms
Speed: 1.2ms preprocess, 34.8ms inference, 1.2ms postprocess per image at shape (1, 3, 224, 640)
Results saved to runs/detect/predict
